<a href="https://colab.research.google.com/github/HausReport/MightyLogic/blob/master/Notebooks/Turf_War_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hello!

If you're seeing this on GitHub, you can open the document on Google by clicking the "Open in Collab" button above.  Once this document is open in Google, you can see the interactive content by selecting `Runtime` on the menubar and then `Run All.`


# Set up environment


In [111]:
import sys
# need newer plotly/plotly_express than collab provides?
#!{sys.executable} -m pip install --force-reinstall --upgrade "plotly"
#!{sys.executable} -m pip install --force-reinstall --upgrade "plotly_express"

#
# Moving some of the guts to GitHub and refactoring 
#
!{sys.executable} -m pip install --upgrade --force-reinstall "git+https://github.com/HausReport/MightyLogic.git#egg=MightyLogic"

#
# Imports
#
from random import randrange
from scipy.signal import convolve2d
from enum import Enum, auto
from pprint import pprint
import plotly.figure_factory as ff

import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import math

#import sklearn
from sklearn.preprocessing import PowerTransformer, QuantileTransformer
import numpy as np
from sklearn.preprocessing import StandardScaler

from MightyLogic.Currency import Currency as Currency
from MightyLogic.Values import Values as Values
from MightyLogic.TurfWar.Tiles.BuildingFactory import *
from MightyLogic.TurfWar.TurfWarMap import TurfWarMap

#Values.GOLD

  Cloning https://github.com/HausReport/MightyLogic.git to /tmp/pip-install-ec7w78je/mightylogic_edc8b211562f463f9382dc78c55d15ce
  Running command git clone -q https://github.com/HausReport/MightyLogic.git /tmp/pip-install-ec7w78je/mightylogic_edc8b211562f463f9382dc78c55d15ce
  Created wheel for MightyLogic: filename=MightyLogic-0.1-py3-none-any.whl size=7969 sha256=e34045d74f31a1a45539161d10ec98a2e20182fd55e83c7a266aee800ddd0747
  Stored in directory: /tmp/pip-ephem-wheel-cache-fpa_q3mr/wheels/84/7a/55/d74e49b550ce0fddb2379db726853920650df706dedea8daae
Successfully built MightyLogic
  Attempting uninstall: MightyLogic
    Found existing installation: MightyLogic 0.1
    Uninstalling MightyLogic-0.1:
      Successfully uninstalled MightyLogic-0.1


# Turf War Demo

In [112]:

#
# A Turf War Map
#

map2 =TurfWarMap()

# Use convenience functions to build value array
map2.addTile('A',1, mountains(3000,2000))
map2.addTile('A',2, forest(3000,0,2000))
map2.addTile('A',3, plains(3000, 2000))
map2.addTile('A',4, mountains(1600,1000))
map2.addTile('A',5, forest(3000,0,2000))
map2.addTile('A',6, plains(3000, 2000))

map2.addTile('B',1, factory(1650,500,5000))
map2.addTile('B',2, mountains(5000,5000))
map2.addTile('B',3, lighthouse(3500,300,7000))
map2.addTile('B',4, plains(5000,5000))
map2.addTile('B',5, mountains(5000,5000))
map2.addTile('B',6, gold_mine(90000,500,5000))

map2.addTile('C',1, forest(3000,0,2000))
map2.addTile('C',2, plains(5000,5000))
map2.addTile('C',3, mountains(5000,5000))
map2.addTile('C',4, forest(5000,0,5000))
map2.addTile('C',5, hells_gate(600,800,8500))
map2.addTile('C',6, mountains(3000,2000))

map2.addTile('D',1, mountains(3000,2000))
map2.addTile('D',2, witch_hut(3500,600, 7000))
map2.addTile('D',3, plains(5000,5000))
map2.addTile('D',4, altar(600,800,8500))
map2.addTile('D',5, forest(5000,0,5000))
map2.addTile('D',6, lighthouse(2000, 400, 5000))

map2.addTile('E',1, plains(3000, 2000))
map2.addTile('E',2, mountains(3000,2000))
map2.addTile('E',3, forest(3000,0,2000))
map2.addTile('E',4, plains(3000, 2000))
map2.addTile('E',5, mountains(3000,2000))
map2.addTile('E',6, forest(3000,0,2000))


# Payout array
image = map2.getValues()
res = map2.stagingScores()
#pprint(image)

In [113]:
#
# An approach that shows well-labeled heatmaps.
#

def get_heatmap(arr, title, round=True):
  flip = np.flip( np.array(arr), axis=0)  # use numpy to flip array vertically
  z = flip
  if round:
    z = z.round()

  x = ['1', '2', '3', '4','5','6']
  y = ['E','D','C','B','A']

  #z_text = ["%.0f" % number for number in z] - doesn't work

  if round:
    z_text = flip.round( decimals=-1)
  else:
    z_text = flip.round( decimals=3)
  # Possible to build up z_text to make it more friendly
  # z_text = [['Win', 'Lose', 'Win'],
  #          ['Lose', 'Lose', 'Win'],
  #          ['Win', 'Win', 'Lose']]

  fig = ff.create_annotated_heatmap(z, x=x, y=y, annotation_text=z_text, colorscale='Viridis')
  fig.update_layout(title=title, autosize=False,
                  width=500, height=500,
                  margin=dict(l=65, r=50, b=65, t=90))
  return fig #fig.show()



In [114]:
#
# The payout array can be accessed as a Pandas dataframe.
#
payout = map2.getDataFrame()
payout

,1,2,3,4,5,6
A,30003.0,30003.0,30003.0,15001.6,30003.0,30003.0
B,82582.5,75005.0,109535.0,75005.0,75005.0,136500.0
C,30003.0,75005.0,75005.0,75005.0,139560.0,30003.0
D,30003.0,114035.0,75005.0,139560.0,75005.0,81020.0
E,30003.0,30003.0,30003.0,30003.0,30003.0,30003.0


In [115]:
#
# The staging array can be accessed as a Pandas dataframe.
#
staging =map2.getStrategicDataFrame()
staging

,1,2,3,4,5,6
A,79.767369,0.000000,51.275865,0.000000,12.373764,61.782278
B,0.000000,121.035331,0.000000,65.405625,99.132865,0.000000
C,79.767369,81.720436,70.403210,130.893111,0.000000,139.174832
D,108.361429,0.000000,131.633495,0.000000,121.853415,0.000000
E,0.000000,59.061723,0.000000,59.622059,12.373764,42.316601


In [116]:
#
# Heatmaps show the best tiles for payout round and non-payout rounds.
#


fig1 = get_heatmap(image,"Payout Values for 3rd Rounds")
fig2 = get_heatmap(res,"Strategic Values for Non-Payout Rounds")

#fig = go.Figure()
#fig.add_traces([fig1.data[0], fig2.data[0]])
#fig.layout.update(fig1.layout)
#fig.layout.update(fig2.layout)

fig1.show()
fig2.show()

In [117]:
#
# Shows payout shares for holding a tile on round 3n 
#

gm = map2.getTile('B',6) #b6 # c5 # gold_mine(90000,500,5000)
print(gm.payouts())


Payouts for B:6 Gold Mine
Place               Reward
First               6,300 gold, 350 contribution
2nd                 5,400 gold, 300 contribution
3rd                 4,500 gold, 250 contribution
4-6                 3,600 gold, 200 contribution
7-11                2,700 gold, 150 contribution
12-25               1,800 gold, 100 contribution
26-44               900 gold, 50 contribution



In [118]:
#
# List of buildings easy to parse by players.
#
map2.printBuildingList()

Building          Points      For 1st Place                      Guild Gets
C-5: Hell's Gate  139,560      42 epic souls, 595 contribution,  800 influence.
D-4: Altar        139,560      42 epic souls, 595 contribution,  800 influence.
B-6: Gold Mine    136,500         6,300 gold, 350 contribution,  500 influence.
D-2: Witch Hut    114,035     245 rare souls, 490 contribution,  600 influence.
B-3: Lighthouse   109,535     245 rare souls, 490 contribution,  300 influence.
B-1: Factory       82,582      116 soul dust, 350 contribution,  500 influence.
D-6: Lighthouse    81,020     140 rare souls, 350 contribution,  400 influence.


# Analytical Solution to Find Next Move?

In [119]:
def scale(target):
  scaler = StandardScaler().fit(target)
  X_scaled = scaler.transform(target)
  X_scaled = 1 * (X_scaled + 1)
  return(X_scaled)

In [120]:
#
# High values are desirable payout locations, low are less desirable
#
scaled_payout = scale(payout)
pd.DataFrame(scaled_payout)

,0,1,2,3,4,5
0,0.5,-0.094801,-0.114596,-0.193130,0.007649,0.256737
1,3.0,1.320660,2.499776,1.185935,1.126549,2.769380
2,0.5,1.320660,1.364708,1.185935,2.731603,0.256737
3,0.5,2.548282,1.364708,2.669610,1.126549,1.460409
4,0.5,-0.094801,-0.114596,0.151649,0.007649,0.256737


In [121]:
#
# High values are desirable staging locations, low are less desirable
#
scaled_staging = scale(staging)
pd.DataFrame(scaled_staging)

,0,1,2,3,4,5
0,1.582269,-0.111034,1.012479,-0.050648,0.276166,1.256068
1,-0.191285,2.457059,-0.030725,1.291922,1.983918,0.050933
2,1.582269,1.622887,1.401623,2.636172,0.032603,2.765698
3,2.218032,-0.111034,2.647348,-0.050648,2.431146,0.050933
4,-0.191285,1.142121,-0.030725,1.173203,0.276166,0.876367


In [122]:
#
# Highest values are desirable staging locations, lowest are desirable payout locations
#
pd.DataFrame(scaled_staging-scaled_payout)

,0,1,2,3,4,5
0,1.082269,-0.016233,1.127074,0.142481,0.268517,0.999331
1,-3.191285,1.136399,-2.530501,0.105986,0.857369,-2.718446
2,1.082269,0.302227,0.036915,1.450237,-2.699000,2.508961
3,1.718032,-2.659315,1.282640,-2.720259,1.304596,-1.409476
4,-0.691285,1.236922,0.083871,1.021554,0.268517,0.619630


In [123]:
moves = scaled_staging-scaled_payout

np.set_printoptions(edgeitems=10)
np.core.arrayprint._line_width = 250
get_heatmap(moves,"Movement Array", round=False).show()
moves

array([[ 1.08226936, -0.01623281,  1.12707442,  0.14248141,  0.26851741,
         0.99933057],
       [-3.19128549,  1.13639928, -2.53050105,  0.10598634,  0.85736873,
        -2.71844604],
       [ 1.08226936,  0.3022271 ,  0.03691516,  1.45023663, -2.69899986,
         2.50896105],
       [ 1.71803226, -2.65931546,  1.28264045, -2.72025859,  1.30459629,
        -1.4094758 ],
       [-0.69128549,  1.23692188,  0.08387103,  1.02155421,  0.26851741,
         0.61963023]])

In [136]:
#flip = np.flip( moves, axis=1) 
flip = np.flip( moves, axis=1) 
fig = go.Figure(data=[go.Surface(z=flip)])

fig.update_layout(title='Mt Bruno Elevation', autosize=False,
                  width=500, height=500,
                  margin=dict(l=65, r=50, b=65, t=90))

fig.show()

In [139]:
from sklearn import preprocessing

u,v= np.gradient(np.array(flip))
fig1 = get_heatmap(u,"x-component of gradient", round=False)
fig2 = get_heatmap(v,"y-component of gradient", round=False)

#fig = go.Figure()
#fig.add_traces([fig1.data[0], fig2.data[0]])
#fig.layout.update(fig1.layout)
#fig.layout.update(fig2.layout)

pd.DataFrame(u)
#fig1.show()
#fig2.show()

,0,1,2,3,4,5
0,-3.717777,0.588851,-0.036495,-3.657575,1.152632,-4.273555
1,0.754815,-1.483759,0.653878,-0.545080,0.159230,0.000000
2,0.654485,0.223614,-1.413122,1.906571,-1.897857,2.454659
3,-0.944665,1.483759,-0.214341,0.023478,0.467347,-0.886777
4,2.029106,-1.036079,3.741813,-1.198769,3.896237,-2.409318


In [126]:
from scipy.interpolate import interp2d
move_fx = interp2d([0,1,2,3,4,5],[0,1,2,3,4], u)
move_fy = interp2d([0,1,2,3,4,5],[0,1,2,3,4], v)

move_fx(1.3,1.3)[0]
move_fx(0,0)[0]

2.4093177416479605

In [127]:
moves.shape

(5, 6)

In [134]:
def show_gradient(farr):
  #
  # Flipping vertically makes sense - converting from arr index to y value
  #
  flip = np.flip( farr, axis=1)  #flip rows with axis=1
  grad_target = flip
  #grad_target = farr

  #
  # Do the gradient, results are ordered by axis number, 
  # "The first one is the gradient of all the row values 
  #  and the second one is the gradient along the column."
  #
  gx,gy = np.gradient(1* np.array(grad_target), .1, .1)


  # y in [0..4], x in [0..5]
  y0,y1 = -0.5,4.5
  x0,x1 = -0.5, 5.5
  n = 60
  #delta = .1
  x = np.arange(x0, x1, (x1-x0)/n)
  y = np.arange(y0, y1, (y1-y0)/n)

  # Linear interpolation functions of gx, gy
  delx = interp2d([0,1,2,3,4,5],[0,1,2,3,4], gx)
  dely = interp2d([0,1,2,3,4,5],[0,1,2,3,4], gy)

  # apply functions to get u and v vectors
  u = delx(x, y)
  v = dely(x, y)

  fig = ff.create_streamline(x, y, u,v, arrow_scale=.1, name='streamline')
  #fig = ff.create_quiver(xnew, ynew, zxnew, zynew, scale=.1) #, x_start=0, x_end=6)

  # add dots at integers
  fig.add_trace(go.Scatter(x=[0,0,0,0,0,1,1,1,1,1,2,2,2,2,2,3,3,3,3,3,4,4,4,4,4,5,5,5,5,5], y=[0,1,2,3,4,0,1,2,3,4,0,1,2,3,4,0,1,2,3,4,0,1,2,3,4,0,1,2,3,4],
                          mode='markers',
                          marker_size=14,
                          name='source point'))
  #return grad[1]
  fig.show()


In [135]:
show_gradient(moves)


In [130]:
y0,y1 = 0.0,4.5
x0,x1 = 0.0, 5.5
n = 12
xnew = np.arange(x0, x1, (x1-x0)/n)
ynew = np.arange(y0, y1, (y1-y0)/n)
v = move_fx(xnew, ynew)
u = move_fy(xnew, ynew)
#fig = go.Figure(data=[go.Surface(z=zynew, x=xnew, y=ynew)])
#fig.show()

# Create streamline figure
fig = ff.create_streamline(xnew, ynew,u, v, name='streamline')
fig.add_trace(go.Scatter(x=[0,0,0,0,0,1,1,1,1,1,2,2,2,2,2,3,3,3,3,3,4,4,4,4,4,5,5,5,5,5], y=[0,1,2,3,4,0,1,2,3,4,0,1,2,3,4,0,1,2,3,4,0,1,2,3,4,0,1,2,3,4],
                          mode='markers',
                          marker_size=14,
                          name='source point'))
fig.show()


# Notes


1.   Annotated heatmap: https://plotly.com/python/annotated-heatmap/
2.   A-Star search on grid in python at https://github.com/BaijayantaRoy/Medium-Article/blob/master/A_Star.ipynb

